In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os


In [2]:
input_file="diy_rejection_28-April'25.csv"
output_file="diy_mco_rejection_27th April'25.csv"

In [3]:
#input_file="diy_all_rejection_01st April.csv"
#output_file="diy_mco_all_rejection_31st Mar'25.csv"

In [4]:
rejection_data=pd.read_csv(input_file,low_memory=False,memory_map=True)
len(rejection_data)

22132

In [5]:
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1298300733,a1810b44-5e37-475c-9036-0b6951093a2c,PROPRIETORSHIP,Random document uploaded,BankStatement,2025-04-14 14:10:25.000,diy_mco,pan_addition,NaN
1,1467894069,b3aed953-c047-4d0d-a2ee-f541a872e40e,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,bankPassbook,2025-04-14 19:59:26.000,diy_mco,pan_addition,NaN
2,1944070983,41ab21e1-04a8-4e3d-9340-ad74f0f64968,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,BankStatement,2025-04-09 08:39:36.000,diy_mco,NaN,NaN
3,1944070983,41ab21e1-04a8-4e3d-9340-ad74f0f64968,INDIVIDUAL,Savings account - PD name does not match with...,bankVerificationStatus,2025-04-09 08:39:36.000,diy_mco,NaN,NaN
4,1837603175,d769ca28-0d63-409e-b77b-fed88f5e9a78,INDIVIDUAL,Name on cheque/bank statement mismatch with ba...,BankStatement,2025-04-08 10:15:03.000,diy_mco,NaN,NaN


In [6]:
rejection_data['updated_at']=pd.to_datetime(rejection_data['updated_at'],errors='coerce')
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1298300733,a1810b44-5e37-475c-9036-0b6951093a2c,PROPRIETORSHIP,Random document uploaded,BankStatement,2025-04-14 14:10:25,diy_mco,pan_addition,NaN
1,1467894069,b3aed953-c047-4d0d-a2ee-f541a872e40e,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,bankPassbook,2025-04-14 19:59:26,diy_mco,pan_addition,NaN
2,1944070983,41ab21e1-04a8-4e3d-9340-ad74f0f64968,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,BankStatement,2025-04-09 08:39:36,diy_mco,NaN,NaN
3,1944070983,41ab21e1-04a8-4e3d-9340-ad74f0f64968,INDIVIDUAL,Savings account - PD name does not match with...,bankVerificationStatus,2025-04-09 08:39:36,diy_mco,NaN,NaN
4,1837603175,d769ca28-0d63-409e-b77b-fed88f5e9a78,INDIVIDUAL,Name on cheque/bank statement mismatch with ba...,BankStatement,2025-04-08 10:15:03,diy_mco,NaN,NaN


In [7]:
current_date = pd.Timestamp(datetime.now().date())
current_date


Timestamp('2025-04-28 00:00:00')

In [8]:
rejection_data=rejection_data[rejection_data['updated_at'].dt.date != current_date]
len(rejection_data)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


21673

In [9]:
rejection_data=rejection_data[rejection_data['solution_type_level_2'].isin(['diy_upgrade','']) | rejection_data['solution_type_level_2'].isna()]
len(rejection_data)

11453

In [10]:
rejection_data.loc[~rejection_data['entity_type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'entity_type']='NON_PROPRIETORSHIP'
rejection_data['entity_type'].value_counts()

INDIVIDUAL            6824
PROPRIETORSHIP        4595
NON_PROPRIETORSHIP      34
Name: entity_type, dtype: int64

In [11]:
rejection_data.insert(rejection_data.columns.get_loc('entity_type')+1,'entuty_type_1',np.nan)
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [12]:
rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']
rejection_data['entuty_type_1'].value_counts()

INDIVIDUAL            6824
PROPRIETORSHIP        4595
NON_PROPRIETORSHIP      34
Name: entuty_type_1, dtype: int64

In [13]:
rejection_data['entuty_type_1']=rejection_data['entuty_type_1'].replace(['PROPRIETORSHIP','NON_PROPRIETORSHIP'],'NON_EDC')
rejection_data['entuty_type_1'].value_counts()

INDIVIDUAL    6824
NON_EDC       4629
Name: entuty_type_1, dtype: int64

In [14]:
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [15]:
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_MCO'
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (~rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_ReKYC'
rejection_data.loc[~(rejection_data['entity_type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [16]:
rejection_data.to_csv(output_file, index=False)